In [20]:
import numpy as np
import pandas as pd
import json 
from datetime import datetime  # Correct import for current datetime
import charset_normalizer
import os



def search_duplicates(data, exclude_land_duplicates=True):
    duplicates = data[data.duplicated(['name', 'maybeboard'], keep=False)]

    if exclude_land_duplicates:
        duplicates = duplicates[~duplicates['Type'].str.contains('Land', case=False)]

    # Reset the "Select" column
    data["Select"] = False
    
    data.loc[data.index.isin(duplicates.index), "Select"] = True

    # Select all duplicate for each group
    #for name, group in duplicates.groupby(['name', 'maybeboard']):
    #    #st.session_state.data.loc[group.index[1:], "Select"] = True  # Select all entries
    #    st.session_state.data.loc[group.index[0:], "Select"] = True  # Select all entries
    
    data.sort_values(by=['Select','name'])

    return data

# Function to load local JSON data
def local_to_jsondata(json_file_path):    
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Function to convert JSON data to a filtered DataFrame
def jsondata_to_filtered_df(data):
    normalized_df = pd.json_normalize(data['cards']['mainboard'])
    columns_to_display = [
        'details.name', 'cmc', 'type_line', 'colors', 'details.set', 'details.collector_number', 
        'details.rarity', 'colorCategory', 'status', 'finish', 'board', 
        'imgUrl', 'imgBackUrl', 'tags', 'notes', 'details.mtgo_id'
    ]
    available_columns = [col for col in columns_to_display if col in normalized_df.columns]
    filtered_df = normalized_df[available_columns].copy()
    missing_columns = [col for col in columns_to_display if col not in normalized_df.columns]
    for col in missing_columns:
        filtered_df.insert(0, col, value="")
    rename_columns = {
        'details.name': 'name', 'cmc': 'CMC', 'type_line': 'Type', 'colors': 'Color',
        'details.set': 'Set', 'details.collector_number': 'Collector Number', 'details.rarity': 'Rarity',
        'colorCategory': 'Color Category', 'status': 'status', 'finish': 'Finish',
        'board': 'maybeboard', 'imgUrl': 'image URL', 'imgBackUrl': 'image Back URL',
        'tags': 'tags', 'notes': 'Notes', 'details.mtgo_id': 'MTGO ID'
    }
    filtered_df.rename(columns=rename_columns, inplace=True)
    fill_columns = {
        'CMC': 'details.cmc', 'Color Category': 'details.colorcategory', 'Type': 'details.type', 
        'Rarity': 'details.rarity', 'image URL': 'imgUrl', 'image Back URL': 'imgBackUrl', 
        'Color': 'details.colors'
    }
    for new_col, old_col in fill_columns.items():
        if old_col in normalized_df.columns:
            filtered_df[new_col] = filtered_df[new_col].fillna(normalized_df[old_col])
    filtered_df['CMC'] = pd.to_numeric(filtered_df['CMC'], errors='coerce').fillna(0).astype(int)
    filtered_df['tags'] = filtered_df['tags'].fillna('').apply(lambda x: x if isinstance(x, list) else [])
    filtered_df.loc[filtered_df['Finish'].isnull(), 'Finish'] = 'Non-foil'
    filtered_df.loc[filtered_df['maybeboard'].isnull(), 'maybeboard'] = ''
    filtered_df.loc[filtered_df['Notes'].isnull(), 'Notes'] = ''
    filtered_df.loc[filtered_df['image URL'].isnull(), 'image URL'] = ''
    filtered_df.loc[filtered_df['image Back URL'].isnull(), 'image Back URL'] = ''
    columns_oder = [
        'name', 'CMC', 'Type', 'Color', 'Set', 'Collector Number', 'Rarity', 'Color Category', 
        'status', 'Finish', 'maybeboard', 'image URL', 'image Back URL', 'tags', 'Notes', 'MTGO ID'
    ]
    filtered_df = filtered_df.reindex(columns=columns_oder)
    return filtered_df


filtered_df = pd.DataFrame(columns=[
        'name', 'CMC', 'Type', 'Color', 'Set', 'Collector Number', 'Rarity', 'Color Category', 
        'status', 'Finish', 'maybeboard', 'image URL', 'image Back URL', 'tags', 'Notes', 'MTGO ID'
    ])

def load_json_file(json_file, verbose = 0):

    json_data = json.load(json_file)
    new_df = jsondata_to_filtered_df(json_data)
    if verbose >0:
        print("new df")
        display(new_df)
    return new_df
    

def load_csv_file(csv_file, encoding_value=None, verbose = 0):
# Upload CSV file and inspect before concatenation
    if encoding_value is None:
        encoding_value = "utf-8"
    print("encoding value use", encoding_value)
    print("Upload CSV File")
    if csv_file is not None:
        #csv_df = pd.read_csv(csv_file, encoding="windows-1250")
        csv_df = pd.read_csv(csv_file, encoding=encoding_value)

        if verbose >0:
            display(csv_df)
    return csv_df

def get_encoding_of_csv(csv_file):
    # look at the first ten thousand bytes to guess the character encoding
    with open(csv_file, 'rb') as rawdata:
        result = charset_normalizer.detect(rawdata.read(40000))

    # check what the character encoding might be
    return result

def find_and_del_X(df):

    df = df[df.name !="X" ]
    
    return df


# Function to compare rows and decide which to keep
def handle_duplicates(df):
    deleted_entries = []

    if "Select" not in df.columns:
        df["Select"] = "False"
    
    duplicates = df.loc[df['Select'] == True]
    
    for name in duplicates['name'].unique():
        duplicate_entries = duplicates[duplicates['name'] == name]
 
        # Display only the differing attributes for user input
        differing_columns = duplicate_entries.loc[:, duplicate_entries.apply(pd.Series.nunique) != 1].columns
        duplicate_entries_display = duplicate_entries[differing_columns].reset_index(drop=True)
        print(f"Duplicate entries for {name}:")
        print(duplicate_entries_display)
        selection_input = input(f"Select the entries to keep for {name} (e.g., '1 2'):")
        
      
        try:
            selections = list(map(int, selection_input.split()))
            indices_to_keep = [duplicate_entries.index[i] for i in selections]
            print(f"DUplicates entries  {duplicate_entries}")
            print(F"indices to keep {indices_to_keep}")
            
            indices_to_delete = [idx for idx in duplicate_entries.index if idx not in indices_to_keep]
            deleted_entries.extend(indices_to_delete)
            df = df.drop(indices_to_delete)
        except (IndexError, ValueError):
            print(f"Invalid input. Dropping all duplicate entries for {name}.")
            deleted_entries.extend(duplicate_entries.index)
            df = df.drop(duplicate_entries.index)
    
    return df, deleted_entries
    

# Define the function to add space after comma if not present
def add_space_after_comma(name):
    return ', '.join([part.strip() for part in name.split(',')])




In [2]:
import json
# Path to the JSON file
json_file_path = r"C:\Users\felix\Documents\MtG-OCR\MtG-OCR\data\Card_Identification\config\default-cards-20240704210810.json"

# Load the JSON data
with open(json_file_path, 'r', encoding='utf-8') as file:
    scryfall_data = json.load(file)

print("rdy")

rdy


In [8]:

def clean_data(data):    
    # Delete X
    data = data[data.name !="X" ]
    data = data[data.name !="Castle" ]
    data["Type"] = data["Type"].fillna('')
    data["name"] = data["name"].fillna('')
    #data.loc[data["Finish"] == ""] = "Non-foil"
    data.loc[data["Finish"] != "Foil", "Finish"] = "Non-foil"
    data.loc[data["status"] != "Owned", "status"] = "Proxied"
    
    
    # Apply the function to the "name" column
    data['name'] = data['name'].apply(add_space_after_comma)
    
    name_to_color = {}    
    # Adjust color categeory
    # Create a mapping of card names to their corresponding colors
    for card in scryfall_data:
        if 'name' in card and 'colors' in card:
            name_to_color[card['name']] = ''.join(card['colors']) if card['colors'] != [] else 'r'

    # Update the Color column in the data DataFrame
    data['Color'] = data['name'].apply(lambda x: name_to_color.get(x, ''))
    data.loc[data["Color"] == 'c', "Color"] = ""
    #???
    data['Color Category'] = data['name'].apply(lambda x: name_to_color.get(x, ''))
    #if type  == adventure color category = color

    land_mask = data['Type'].str.contains("Land", na=False)    
    land_to_color ={}
    for card in scryfall_data:
        if 'name' in card and 'colors' in card:
            land_to_color[card['name']] = ''.join(card['color_identity']) 


    # Update the Color column for only Land rows using the name_to_color mapping
    data.loc[land_mask, 'Color Category'] = 'l'
    data.loc[land_mask, 'Color'] = data.loc[land_mask, 'name'].apply(lambda x: land_to_color.get(x, '') if not pd.isna(x) else '')

    data["Color"] = data["Color"].fillna("")
    data["Color Category"] = data["Color Category"].fillna("c")

 
            
    # Filter the DataFrame for rows where 'Type' contains 'Adventure'
    adventure_mask = data["Type"].str.contains("Adventure")
    
    # Update the 'name' column for the filtered adventure_mask
    data.loc[adventure_mask, "name"] = data.loc[adventure_mask, "name"].str.split("//").str[0]
    data.loc[adventure_mask, "Type"] = data.loc[adventure_mask, "Type"].str.split("//").str[0]
  
    # Create a mapping of card names to their corresponding layout
    name_to_layout = {card['name']: card['layout'] for card in scryfall_data}
    
    # Function to check if the card name contains "//" and has a "transform" layout in Scryfall data
    def should_split(name):
        return "//" in name and name_to_layout.get(name, "") == "transform"
    
    # Apply the mask to identify relevant rows
    split_mask = data['name'].replace('"','').apply(should_split)
    
    # Update the 'name' and 'Type' columns for the filtered split_mask
    data.loc[split_mask, "name"] = data.loc[split_mask, "name"].str.split("//").str[0]
    data.loc[split_mask, "Type"] = data.loc[split_mask, "Type"].str.split("//").str[0]
   
    #data.loc[data["Finish"] == ""] = "Non-foil"
    data.loc[data["Finish"] != "Foil", "Finish"] = "Non-foil"
    data.loc[data["status"] != "Owned", "status"] = "Proxied"
    #data.loc[data["maybeboard"] != "True", "maybeboard"] = "False"
    
    
    # Check if the 'image_back_url' column exists, if not add it with empty values
    if 'image_back_url' not in data.columns:
        data['image_back_url'] = ""
    
    # Define the desired column order
    desired_columns = [
        "name", "CMC", "Type", "Color", "Set", "Collector Number", "Rarity",
        "Color Category", "status", "Finish", "maybeboard", "image URL", 
        "image_back_url", "tags", "Notes", "MTGO ID"
    ]
    
    
    # Reorder the columns to match the desired order
    data = data[desired_columns]

    return data 


def export_data(data, expo_path=None):
     # Check if expo_path is None
    if expo_path is None:
        # Get the current working directory
        current_location = os.getcwd()
        # Generate the current datetime string
        datetime_str = datetime.now().strftime("%Y%m%d%H%M%S")
        # Create the default export path
        expo_path = os.path.join(current_location, f"cubecobra_export_{datetime_str}.csv")
        expo_path = os.path.join(r"C:\Users\felix\Documents", f"cubecobra_export_{datetime_str}.csv")
        
        print(expo_path)
    if 'Select' in data.columns:
        data = data.drop(columns=["Select"])
           
    # List of columns to add quotes around
    columns_to_quote = ["name", "Collector Number", "Type", "Set", "Notes", "tags"]
    
    # Add quotes around the specified columns if they do not already contain quotes
    for column in columns_to_quote:
        data[column] = data[column].apply(lambda x: '"' + str(x) + '"' if not (isinstance(x, str) and (x.startswith('"') and x.endswith('"'))) else str(x))
    
    data.to_csv(expo_path, index=False)
    
    # Read the exported CSV file
    with open(expo_path, 'r', encoding='windows-1250') as file:
        csv_content = file.read()
          
    # Load the CSV file into a DataFrame
    data = pd.read_csv(expo_path)
        # Convert all columns to string to apply string replacement
    data = data.astype(str)
       
    # Perform modifications
    csv_content = csv_content.replace('"""', '"').replace('nan', '').replace('False', 'false').replace('.0',"").replace(' "','"')
    #print(csv_content)
    
    # Write the modified content back to the CSV file
    with open(expo_path, 'w', encoding='windows-1250') as file:
        file.write(csv_content)


In [20]:
csv_file = r"C:\Users\felix\Downloads\MainCubeMH3_old.csv"
csv_file= r"C:\Users\felix\Documents\MainCubeMH3_old.csv"

infos = get_encoding_of_csv(csv_file)
encoding_value = infos['encoding']


#csv_file = r"C:\Users\felix\Documents\test1.csv"
# Load 
data = load_csv_file(csv_file,"utf-8")
data = clean_data(data)

land_mask = data["name"].str.contains("Blood")
display(data[land_mask])
land_mask = data["name"].str.contains("revea")
display(data[land_mask])

export_data(data)



encoding value use utf-8
Upload CSV File


,name,CMC,Type,Color,Set,Collector Number,Rarity,Color Category,status,Finish,maybeboard,image URL,image_back_url,tags,Notes,MTGO ID
169,Blood Fountain,1,Artifact,B,vow,95,common,B,Owned,Non-foil,False,NaN,,NaN,NaN,94506.0
181,Blood Aspirant,2,Creature - Satyr Berserker,R,thb,128,uncommon,R,Owned,Foil,False,NaN,,NaN,NaN,79380.0
196,"Kellan, the Fae-Blooded",3,Legendary Creature - Human Faerie,,woe,230,mythic,,Owned,Non-foil,False,NaN,,NaN,NaN,116824.0
333,Blood Crypt,0,Land - Swamp Mountain,BR,rna,245,rare,l,Owned,Non-foil,False,NaN,,NaN,NaN,71530.0
339,Blood Crypt,0,Land - Swamp Mountain,BR,rna,245,rare,l,Owned,Foil,False,NaN,,NaN,NaN,71530.0
340,Bloodstained Mire,0,Land,,ktk,230,rare,l,Owned,Non-foil,False,NaN,,NaN,NaN,54198.0
341,Bloodstained Mire,0,Land,,ktk,230,rare,l,Owned,Non-foil,False,NaN,,NaN,NaN,54198.0


,name,CMC,Type,Color,Set,Collector Number,Rarity,Color Category,status,Finish,maybeboard,image URL,image_back_url,tags,Notes,MTGO ID


C:\Users\felix\Documents\cubecobra_export_20240709213543.csv


In [164]:
data_RG = data_RG[data_RG.name !="Aspect of Manticore"]
data_ges = pd.concat([data_RG, data_WBU], ignore_index=True)


adventure_mask = data_ges["name"].str.contains("Sharae")
display(data_ges[adventure_mask])
data_ges = clean_data(data_ges)

data_ges.loc[data_ges["name"] == "Fire","name"] = "Fire // Ice"
data_ges.loc[data_ges["name"] == "Push","name"] = "Push // Pull"
data_ges.loc[data_ges["name"] == "Flotsam","name"] = "Flotsam // Jetsam"
adventure_mask = data_ges["name"].str.contains("Sharae")
display(data_ges[adventure_mask])
adventure_mask = data_ges["name"].str.contains("Blood")
display(data_ges[adventure_mask])
export_data(data_ges)
land_mask = data_ges["name"].str.contains("Blood")
display(data_ges[land_mask])

,name,CMC,Type,Color,Set,Collector Number,Rarity,Color Category,status,Finish,maybeboard,image URL,image_back_url,tags,Notes,MTGO ID,Select
210,Sharae of Numbing Depths,4,Legendary Creature - Merfolk Wizard,U,woe,213,uncommon,m,Proxied,Non-foil,NaN,NaN,,NaN,NaN,116770.0,False


,name,CMC,Type,Color,Set,Collector Number,Rarity,Color Category,status,Finish,maybeboard,image URL,image_back_url,tags,Notes,MTGO ID
210,Sharae of Numbing Depths,4,Legendary Creature - Merfolk Wizard,UW,woe,213,uncommon,UW,Proxied,Non-foil,NaN,NaN,,NaN,NaN,116770.0


,name,CMC,Type,Color,Set,Collector Number,Rarity,Color Category,status,Finish,maybeboard,image URL,image_back_url,tags,Notes,MTGO ID
88,"Kellan, the Fae-Blooded",3,Legendary Creature - Human Faerie,,woe,230,mythic,,Owned,Non-foil,NaN,NaN,,NaN,NaN,116824.0
118,Blood Aspirant,2,Creature - Satyr Berserker,R,thb,128,uncommon,R,Owned,Foil,NaN,NaN,,NaN,NaN,79380.0
155,Blood Fountain,1,Artifact,B,vow,95,common,B,Owned,Non-foil,NaN,NaN,,NaN,NaN,94506.0
321,Blood Crypt,0,Land - Swamp Mountain,BR,rna,245,rare,l,Owned,Non-foil,NaN,NaN,,NaN,NaN,71530.0
323,Bloodstained Mire,0,Land,,ktk,230,rare,l,Owned,Non-foil,NaN,NaN,,NaN,NaN,54198.0
349,Bloodstained Mire,0,Land,,ktk,230,rare,l,Owned,Non-foil,NaN,NaN,,NaN,NaN,54198.0
357,Blood Crypt,0,Land - Swamp Mountain,BR,rna,245,rare,l,Owned,Foil,NaN,NaN,,NaN,NaN,71530.0


C:\Users\felix\Documents\cubecobra_export_20240709210639.csv


,name,CMC,Type,Color,Set,Collector Number,Rarity,Color Category,status,Finish,maybeboard,image URL,image_back_url,tags,Notes,MTGO ID
88,"""Kellan, the Fae-Blooded""",3,"""Legendary Creature - Human Faerie """,,"""woe""","""230""",mythic,,Owned,Non-foil,NaN,NaN,,"""nan""","""nan""",116824.0
118,"""Blood Aspirant""",2,"""Creature - Satyr Berserker""",R,"""thb""","""128""",uncommon,R,Owned,Foil,NaN,NaN,,"""nan""","""nan""",79380.0
155,"""Blood Fountain""",1,"""Artifact""",B,"""vow""","""95""",common,B,Owned,Non-foil,NaN,NaN,,"""nan""","""nan""",94506.0
321,"""Blood Crypt""",0,"""Land - Swamp Mountain""",BR,"""rna""","""245""",rare,l,Owned,Non-foil,NaN,NaN,,"""nan""","""nan""",71530.0
323,"""Bloodstained Mire""",0,"""Land""",,"""ktk""","""230""",rare,l,Owned,Non-foil,NaN,NaN,,"""nan""","""nan""",54198.0
349,"""Bloodstained Mire""",0,"""Land""",,"""ktk""","""230""",rare,l,Owned,Non-foil,NaN,NaN,,"""nan""","""nan""",54198.0
357,"""Blood Crypt""",0,"""Land - Swamp Mountain""",BR,"""rna""","""245""",rare,l,Owned,Foil,NaN,NaN,,"""nan""","""nan""",71530.0


In [15]:
# Open File

csv_file = r"http://localhost:8888/lab/tree/MtGDuplicates_Notebook/CubeCobraCSV_20240522_211938.csv"
csv_file = r"C:\Users\felix\MtGDuplicates_Notebook\CubeCobraCSV_20240521_221115.csv"
csv_file = r"C:\Users\felix\Documents\MtG-OCR\MtG-OCR\data\Card_Identification\results\CubeCobraCSV_20240705_105835.csv"
csv_file = r"C:\Users\felix\Documents\MtG-OCR\MtG-OCR\data\Card_Identification\results\CubeCobraCSV_20240705_105835_RG.csv"

infos = get_encoding_of_csv(csv_file)
encoding_value = infos['encoding']


#csv_file = r"C:\Users\felix\Documents\test1.csv"
# Load 
data = load_csv_file(csv_file,encoding_value = encoding_value)

data = clean_data(data)


data = search_duplicates(data, exclude_land_duplicates=True)
# Handle duplicates and update the data
data, deleted_entries = handle_duplicates(data)
print("handled data")
display(data)
data = data[data.name !="Aspect of Manticore"]

data_RG = data


encoding value use ascii
Upload CSV File
Duplicate entries for Rabbit Battery:
   maybeboard  image URL  tags  Notes
0         NaN        NaN   NaN    NaN
1         NaN        NaN   NaN    NaN


Select the entries to keep for Rabbit Battery (e.g., '1 2'): 0


DUplicates entries                 name  CMC                                  Type Color  Set  \
126  Rabbit Battery    1  Artifact Creature - Equipment Rabbit     R  neo   
127  Rabbit Battery    1  Artifact Creature - Equipment Rabbit     R  neo   

     Collector Number    Rarity Color Category status    Finish  maybeboard  \
126               157  uncommon              R  Owned  Non-foil         NaN   
127               157  uncommon              R  Owned  Non-foil         NaN   

     image URL image_back_url  tags  Notes  MTGO ID  Select  
126        NaN                  NaN    NaN  97246.0    True  
127        NaN                  NaN    NaN  97246.0    True  
indices to keep [127]
handled data


,name,CMC,Type,Color,Set,Collector Number,Rarity,Color Category,status,Finish,maybeboard,image URL,image_back_url,tags,Notes,MTGO ID,Select
0,"Rishkar, Peema Renegade",3,Legendary Creature - Elf Druid,G,aer,122,rare,G,Owned,Foil,NaN,NaN,,NaN,NaN,62795.0,False
1,Chainweb Aracnir,1,Creature - Spider,G,thb,167,uncommon,G,Owned,Foil,NaN,NaN,,NaN,NaN,79458.0,False
2,Timberland Ancient,6,Creature - Treefolk,G,mom,210,common,G,Owned,Foil,NaN,NaN,,NaN,NaN,110306.0,False
3,Experiment One,1,Creature - Human Ooze,G,2x2,146,common,G,Owned,Foil,NaN,NaN,,NaN,NaN,101776.0,False
4,Titanic Brawl,2,Instant,G,rna,146,common,G,Owned,Non-foil,NaN,NaN,,NaN,NaN,71292.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,Imodane's Recruiter,3,Creature - Human Knight,R,woe,229,uncommon,R,Proxied,Non-foil,NaN,NaN,,NaN,NaN,116820.0,False
134,Young Pyromancer,2,Creature - Human Shaman,R,2x2,131,uncommon,R,Owned,Foil,NaN,NaN,,NaN,NaN,101746.0,False
135,"Gut, True Soul Zealot",3,Legendary Creature - Goblin Shaman,R,clb,180,uncommon,R,Owned,Foil,NaN,NaN,,NaN,NaN,100384.0,False
136,Rimrock Knight,2,Creature - Dwarf Knight,R,eld,294,common,R,Owned,Foil,NaN,NaN,,NaN,NaN,NaN,False


In [18]:
csv_file =r"C:\Users\felix\Documents\MtG-OCR\MtG-OCR\data\Card_Identification\results\CubeCobraCSV_20240521_221115 _WBU.csv"
infos = get_encoding_of_csv(csv_file)
encoding_value = infos['encoding']


#csv_file = r"C:\Users\felix\Documents\test1.csv"
# Load 
data = load_csv_file(csv_file,encoding_value = encoding_value)

data = clean_data(data)


data = search_duplicates(data, exclude_land_duplicates=True)
# Handle duplicates and update the data
data, deleted_entries = handle_duplicates(data)
print("handled data")
display(data)


data_WBU = data

encoding value use windows-1250
Upload CSV File
Duplicate entries for Bone Shards:
    status    Finish  maybeboard  image URL  tags  Notes
0  Proxied  Non-foil         NaN        NaN   NaN    NaN
1    Owned      Foil         NaN        NaN   NaN    NaN


Select the entries to keep for Bone Shards (e.g., '1 2'): 0


DUplicates entries             name  CMC     Type Color  Set  Collector Number  Rarity  \
16  Bone Shards    1  Sorcery     B  mh2                76  common   
17  Bone Shards    1  Sorcery     B  mh2                76  common   

   Color Category   status    Finish  maybeboard  image URL image_back_url  \
16              B  Proxied  Non-foil         NaN        NaN                  
17              B    Owned      Foil         NaN        NaN                  

    tags  Notes  MTGO ID  Select  
16   NaN    NaN  90525.0    True  
17   NaN    NaN  90525.0    True  
indices to keep [16]
Duplicate entries for Quick Study:
    status  maybeboard  image URL  tags  Notes
0  Proxied         NaN        NaN   NaN    NaN
1    Owned         NaN        NaN   NaN    NaN


Select the entries to keep for Quick Study (e.g., '1 2'): 1


DUplicates entries             name  CMC     Type Color  Set  Collector Number  Rarity  \
91  Quick Study    3  Instant     U  woe                65  common   
92  Quick Study    3  Instant     U  woe                65  common   

   Color Category   status    Finish  maybeboard  image URL image_back_url  \
91              U  Proxied  Non-foil         NaN        NaN                  
92              U    Owned  Non-foil         NaN        NaN                  

    tags  Notes   MTGO ID  Select  
91   NaN    NaN  116436.0    True  
92   NaN    NaN  116436.0    True  
indices to keep [92]
Duplicate entries for Crack in Time:
   Collector Number  maybeboard  image URL  tags  Notes  MTGO ID
0                16         NaN        NaN   NaN    NaN      NaN
1               336         NaN        NaN   NaN    NaN      NaN


Select the entries to keep for Crack in Time (e.g., '1 2'): 1


DUplicates entries                name  CMC         Type Color  Set  Collector Number Rarity  \
277  Crack in Time    4  Enchantment     W  who                16   rare   
278  Crack in Time    4  Enchantment     W  who               336   rare   

    Color Category   status    Finish  maybeboard  image URL image_back_url  \
277              W  Proxied  Non-foil         NaN        NaN                  
278              W  Proxied  Non-foil         NaN        NaN                  

     tags  Notes  MTGO ID  Select  
277   NaN    NaN      NaN    True  
278   NaN    NaN      NaN    True  
indices to keep [278]
Duplicate entries for Recommission:
   maybeboard  image URL  tags  Notes
0         NaN        NaN   NaN    NaN
1         NaN        NaN   NaN    NaN


Select the entries to keep for Recommission (e.g., '1 2'): 0


DUplicates entries               name  CMC     Type Color  Set  Collector Number  Rarity  \
286  Recommission    2  Sorcery     W  bro                22  common   
287  Recommission    2  Sorcery     W  bro                22  common   

    Color Category status Finish  maybeboard  image URL image_back_url  tags  \
286              W  Owned   Foil         NaN        NaN                  NaN   
287              W  Owned   Foil         NaN        NaN                  NaN   

     Notes   MTGO ID  Select  
286    NaN  104662.0    True  
287    NaN  104662.0    True  
indices to keep [286]
handled data


,name,CMC,Type,Color,Set,Collector Number,Rarity,Color Category,status,Finish,maybeboard,image URL,image_back_url,tags,Notes,MTGO ID,Select
0,Festerleech,1,Creature - Zombie Leech,B,mkm,85,uncommon,B,Owned,Non-foil,NaN,NaN,,NaN,NaN,121648.0,False
1,Urborg Scavengers,3,Creature - Spirit,B,mat,15,rare,B,Owned,Foil,NaN,NaN,,NaN,NaN,109152.0,False
2,Overwhelming Remorse,5,Instant,B,bro,110,common,B,Owned,Foil,NaN,NaN,,NaN,NaN,104838.0,False
3,Pile On,4,Instant,B,mom,122,rare,B,Owned,Foil,NaN,NaN,,NaN,NaN,110090.0,False
4,Forsaken Miner,1,Creature - Skeleton Rogue,B,otj,88,uncommon,B,Proxied,Non-foil,NaN,NaN,,NaN,NaN,124087.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,Ethersworn Canonist,2,Artifact Creature - Human Cleric,W,mma,14,rare,W,Owned,Foil,NaN,NaN,,NaN,NaN,48804.0,False
289,Take Up the Shield,2,Instant,W,otj,34,common,W,Proxied,Non-foil,NaN,NaN,,NaN,NaN,123979.0,False
290,Solitary Sanctuary,3,Enchantment,W,woe,30,uncommon,W,Owned,Foil,NaN,NaN,,NaN,NaN,116352.0,False
291,Glass Casket,2,Artifact,W,eld,15,uncommon,W,Owned,Foil,NaN,NaN,,NaN,NaN,78122.0,False


In [19]:
csv_file = r"C:\Users\felix\MtGDuplicates_Notebook\CubeCobraCSV_20240521_221115.csv"
csv_file = r"C:\Users\felix\Downloads\MainCubeMH3_old.csv"
infos = get_encoding_of_csv(csv_file)
encoding_value = infos['encoding']
print(infos)
encoding_value = "windows-1250"
#csv_file = r"C:\Users\felix\Documents\test1.csv"
# Load 
data = load_csv_file(csv_file,encoding_value)

#data = load_csv_file(csv_file,"utf-8")
data = clean_data(data)


data = search_duplicates(data, exclude_land_duplicates=True)
# Handle duplicates and update the data
data, deleted_entries = handle_duplicates(data)
print("handled data")
display(data)
data = data[data.name !="Aspect of Manticore"]
display(data[data["name"] =="Bomat Courier"])
display(data[data["name"] =="Blood Crypt"])
export_data(data)


#data_RG = data

{'encoding': 'ascii', 'language': 'English', 'confidence': 1.0}
encoding value use windows-1250
Upload CSV File
handled data


,name,CMC,Type,Color,Set,Collector Number,Rarity,Color Category,status,Finish,maybeboard,image URL,image_back_url,tags,Notes,MTGO ID,Select
0,Faerie Guidemother,1,Creature - Faerie,,eld,274,common,,Owned,Foil,False,NaN,,NaN,NaN,NaN,False
1,Law-Rune Enforcer,1,Creature - Human Soldier,W,clu,64,common,W,Owned,Foil,False,NaN,,NaN,NaN,122912.0,False
2,Novice Inspector,1,Creature - Human Detective,W,mkm,29,common,W,Owned,Non-foil,False,NaN,,NaN,NaN,121546.0,False
3,Recruitment Officer,1,Creature - Human Soldier,W,bro,23,uncommon,W,Owned,Foil,False,NaN,,NaN,NaN,104664.0,False
4,Savannah Lions,1,Creature - Cat,W,dmr,24,common,W,Owned,Foil,False,NaN,,NaN,NaN,107537.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,Watery Grave,0,Land - Island Swamp,BU,grn,259,rare,l,Owned,Non-foil,False,NaN,,NaN,NaN,69931.0,False
411,Windswept Heath,0,Land,,ktk,248,rare,l,Owned,Non-foil,False,NaN,,NaN,NaN,54202.0,False
412,Windswept Heath,0,Land,,ktk,248,rare,l,Owned,Non-foil,False,NaN,,NaN,NaN,54202.0,False
413,Wooded Foothills,0,Land,,ktk,249,rare,l,Owned,Non-foil,False,NaN,,NaN,NaN,54200.0,False


,name,CMC,Type,Color,Set,Collector Number,Rarity,Color Category,status,Finish,maybeboard,image URL,image_back_url,tags,Notes,MTGO ID,Select
302,Bomat Courier,1,Artifact Creature - Construct,r,kld,199,rare,r,Owned,Non-foil,False,NaN,,NaN,NaN,62075.0,False


,name,CMC,Type,Color,Set,Collector Number,Rarity,Color Category,status,Finish,maybeboard,image URL,image_back_url,tags,Notes,MTGO ID,Select
333,Blood Crypt,0,Land - Swamp Mountain,BR,rna,245,rare,l,Owned,Non-foil,False,NaN,,NaN,NaN,71530.0,False
339,Blood Crypt,0,Land - Swamp Mountain,BR,rna,245,rare,l,Owned,Foil,False,NaN,,NaN,NaN,71530.0,False


C:\Users\felix\Documents\cubecobra_export_20240710094617.csv


In [110]:
def show_color_distribution(data):
    color_distribution = []

    
    #data.loc[data["Color" == Nan, "Color Category"]  = "c"
    data.loc[data['Color'].isna(), 'Color Category'] = 'c'
    
    color_category = [ 'w','u', 'b',  'r','g','m',"c"]
    
    for color in color_category :
    
    #    color_distribution.append(data[data.Color == color].name.count())
        #color_distribution.append([data["Color Category"] == color].count())
    #    color_distribution.append(data.loc[data["Color Category"] == color_category])
        #print(data.loc[data["Color Category"] == color_category].name.count())
        print(color, data.loc[data["Color Category"] == color].name.count())
    
    
    
    nan_count = data.loc[~data['Color Category'].isin(color_category), 'name'].count()
    print(f"NaN or other: {nan_count}")
    
    
    color = "m"
    print("multicolor")
    print(color, data.loc[data["Color Category"] == color].name.count()-data.loc[data["Type"] == "Land"].name.count())
    
    
        
    data.loc[data["Color Category"] == "b"].name.count()
    print(color_distribution)


shock_fetch = ["Watery Grave", "Overgrown Tomb", "",
              "Poluted Delta", "Flooded Strand", "Windswepth Heath", "Scalding Tarn", "Arid Mesa", "Verdant Catacombs", "Bloodstained Mire", "Marsh Flats"] 

In [22]:
import pandas as pd

# Assuming 'data' is your DataFrame

# Filter for cards where status is "Proxied"
proxied_cards = data[data['status'] == 'Proxied']['name']

# Export to proxied_cards.txt
proxied_cards.to_csv('proxied_cards.txt', index=False, header=False)

# Filter for cards where Finish is "Non-Foil"
nonfoil_cards = data[data['Finish'] == 'Non-foil']['name']
nonfoil_cards = data[data['Finish'] != 'Foil']['name']
display(nonfoil_cards)
# Export to Nonfoil.txt
nonfoil_cards.to_csv('Nonfoil.txt', index=False, header=False)
print("finished")

2        "Novice Inspector"
8             "Topplegeist"
9             "Astrid Peth"
13     "Luminarch Aspirant"
14     "Pollen-Shield Hare"
               ...         
410          "Watery Grave"
411       "Windswept Heath"
412       "Windswept Heath"
413      "Wooded Foothills"
414      "Wooded Foothills"
Name: name, Length: 273, dtype: object

finished
